In [4]:
pip install langchain-community

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_classic-1.0.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached langsmith-0.5.0-py3-none-any.whl.metadata (15 kB)
  Using cached httpx_sse-0.4.3-py3-none-any.whl.metadata (9.7 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached langchain_text_splitters-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached uuid_utils-0.12.0-cp39-abi3-win_amd64.whl.metadata (1.1 kB)
  Using

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [8]:
pip install langchain-ollama


Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_ollama-1.0.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached ollama-0.6.1-py3-none-any.whl.metadata (4.3 kB)
Using cached langchain_ollama-1.0.1-py3-none-any.whl (29 kB)
Using cached ollama-0.6.1-py3-none-any.whl (14 kB)

   -------------------- ------------------- 1/2 [langchain-ollama]
   ---------------------------------------- 2/2 [langchain-ollama]

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install langchain-core

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
class RAGService:
    def __init__(self, llm_model : str = 'llama3', embedding_model : str = 'llama3'):
        self.llm_model = llm_model
        self.embedding_model = embedding_model

        # Initialize emebedding and LLM
        self.embeddings = OllamaEmbeddings(model=self.embedding_model)
        self.llm = ChatOllama(model=self.llm_model)

        # Vector store 
        self.vector_store = None
        self.retriever = None
        self.chain = None 

    def process_url(self, url : str):
        try:
            loader = WebBaseLoader(url)
            docs = loader.load()
        except:
            raise ValueError(f"Could not load the URL: {url}")

        
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size = 1000,
            chunk_overlap = 200
        )

        splits = text_splitter.split_documents(docs)
    

        if self.vector_store:
            self.vector_store.delete_collection()
        self.vector_store = Chroma.from_documents(
            documents=splits,
            embedding=self.embeddings,
            collection_name='current_doc'
        )
    
        self.retriever = self.vector_store.as_retriever()
        self._create_chain()

        return len(splits)
    
    # 
    def _create_chain(self):
        template = """
                # prompt
                {context}
                {question}
                   """
        prompt = ChatPromptTemplate(template)

        def format_docs(docs):
            return "\n\n".join(doc.page_content for doc in docs)

        self.chain = (
            {'context': self.retriever | format_docs, "question" : RunnablePassthorugh()}
            | prompt
            | self.llm
            | StrOutputParser()
        )
    
    def ask_question(self, question : str) -> str:
        if not self.chain:
            return "Please process a ..."
        return self.chain.invoke(question)
    

rag_server = RAGService()

